# 다중선형회귀
## 다중선형회귀 수행
### 1) 패키지 준비

In [ ]:
import sys
sys.path.append('../../')
import helper

from pandas import read_excel, DataFrame, merge
from matplotlib import pyplot as plt
import seaborn as sb
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelChcekpoint

### 2) 데이터셋 준비

In [ ]:
origin = read_excel('https://data.hossam.kr/F02/fish2.xlsx')
origin.info()

### 3) 데이터 전처리
- 결측치 확인

In [ ]:
origin.isna().sum()

### 4) 탐색적 데이터 분석
- 기본 통계 확인

In [ ]:
origin.describe()

> 평균 값의 차이가 크기 때문에 표준화가 필요해 보인다

- 상자그림

In [ ]:
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 12
plt.rcParams['axes.unicode_minus'] = False

plt.figure(figsize=(10, 5))
sb.boxplot(data=origin)
plt.show()
plt.close()

> 데이터의 분포가 상이하므로 표준화가 필요해 보인다
- 산점도 그래프와 추세선 확인

In [ ]:
plt.figure(figsize = (30, 30))
sb.pairplot(origin)
plt.show()
plt.close()

> `다중공선성`이 발생할 것으로 보이고, 선형회귀보다는 `다항회귀가 더 적합해 보이지만` 예제를 위해 다항선형회귀로 진행하기로 함

### 5) 데이터셋 분할
- 랜덤시드 고정

In [1]:
np.random.seed(777)

NameError: name 'np' is not defined

- 훈련 데이터(독립변수)와 레이블(종속변수) 구분하기

In [ ]:
x = origin[['길이', '높이', '두께']]
y = origin[['무게']]
x.shape, y.shape

- 데이터 표준화

In [ ]:
x_scaler = StandardScaler()
x_scale = x_scaler.fit_transform(x)

y_scaler = StandardScaler()
y_scale = y_scaler.fit_transform(y)

x_scale.shape, y_scale.shape

- 표준화 전/후 상자그림 비교

In [ ]:
# 임시로 독립변수와 종속변수를 하나의 DataFrame으로 병합
x_scale_df = DataFrame(x_scale, columns=['길이', '높이', '두께'])
y_scale_df = DataFrame(y_scale, columns=['무게'])
tmp = merge(x_scale_df, y_scale_df, left_index=True, right_index=True)

fig, ax = plt.subplots(1, 2, figsize = (15, 5), dpi=150)
sb.boxplot(data=origin, ax=ax[0])
sb.boxplot(data=tmp, ax=ax[1])
ax[0].set_title('before')
ax[1].set_title('bafter')
plt.show()
plt.close()

- 훈련 데이터와 검증 데이터 분할

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_scale, 
                                                    y_scale,
                                                    test_size=0.3,
                                                    random_state=777)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

### 6) 모델 개발
- 모델 정의
    - 데이터가 단순하고 개수가 적다면 `과대적합이 발생할 확률이 높기 때문에` 주의하면서 적절한 크기와 모델층을 쌓아야 한다

In [ ]:
my_model = Sequential()

# 1차원의 데이터를 입력으로 받고, 64개의 출력을 가지는 첫 번째 Dense 층
my_model.add(Dense(64, activation='relu', input_shape=(len(x_train[0]), )))
my_model.add(Dense(32, activation ='relu'))

# 하나의 값을 출력 정답의 범위가 정해지지 않기 때문에 활성화 함수는 linear
my_model.add(Dense(1, activation = 'linear'))

my_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
my_model.summary()

- 학습하기

In [ ]:
result = my_model.fit(x_train, 
                      y_train, 
                      epochs=500, 
                      validation_data = (x_test, y_test),
                      callbacks=[EarlyStopping(monitor='val_loss',
                                               patience=10,
                                               verbose=1),
                                 ReduceLROnPlateau(monitor='val_loss',
                                                   patience=3,
                                                   factor=0.5,
                                                   min_lr=0.001,
                                                   verbose=1)])

### 7) 학습 결과 평가

In [ ]:
# 학습 결과
result_df = DataFrame(result.history)
result_df['epochs'] = result_df.index+1
result_df.set_index('epochs', inplace=True)
result_df

In [ ]:
# 그래프 기본설정
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 12
plt.rcParams['axes.unicode_minus'] = False

# 그래프를 그리기 위한 객체 생성
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5), dpi=150)

# 1) 훈련 및 검증 손실 그리기
sb.lineplot(x=result_df.inex,
            y='loss',
            data=result_df,
            color='blue',
            lable='훈련 손실률',
            ax=ax1)
sb.lineplot(x=result_df.index, 
            y_'val_loss',
            data=result_df,
            color='orange',
            label='검증 손실률',
            ax = ax1)
ax1.set_title('훈련 및 검증 손실률')
ax1.set_xlabel('반복회차')
ax1.set_ylabel('손실률')
ax1.grid()
ax1.legend()

# 2) 훈련 및 검증 절대오차 그리기
sb.lineplot(x=result_df.index, 
            y='mae',
            data=result_df, 
            color='blue',
            label='훈련 절대오차',
            ax=ax2)
sb.lineplot(x=result_df.index, 
            y='val_mae',
            data=result_df, 
            color='orange',
            label='검증 절대오차',
            ax=ax2)
ax2.set_title('훈련 및 검증 절대오차')
ax2.set_xlabel('반복회차')
ax2.set_ylabel('정확도')
ax2.grid()
ax2.legend()

plt.show()
plt.close()

- 모델 성능 평가

In [3]:
evaluate1 = my_model.evaluate(x_train, y_train)
print('최종 훈련 손실률: %f, 최종 훈련 절대오차: %f' % (evaluate1[0], evaluate1[1]))
evaluate2 = my_model.evaluate(x_test, y_test)
print('최종 검증 손실률: %f, 최종 검증 절대오차: %f' % (evaluate2[0], evaluate2[1]))

NameError: name 'my_model' is not defined

### 8) 학습 결과 적용
- 예측 결과 산정

In [4]:
train_pred = my_model.predict(x_train)
test_pred = my_model.predict(x_test)

NameError: name 'my_model' is not defined

- 결과 데이터셋 구성

In [ ]:
train_df = DataFrame(x_train, columns=['길이', '높이', '두께'])
train_df['무게(관측치)'] = y_train.flatten()
train_df['무게(예측치)'] = train_pred.flatten()
train_df['훈련오차'] = train_df['무게(관측치)']-train_df['무게(예측치)'] 
train_df.head()

test_df = DataFrame(x_test, columns=['길이', '높이','두께'])
test_df['무게(관측치)'] = y_test.flatten()
test_df['무게(예측치)'] = test_pred.flatten()

test_df['훈련오차'] = test_df['무게(관측치)']-test_df['무게(에측치)']
test_df.head()                                    

- 실제 결과값과 머신러닝에 의한 예측값 비교

In [5]:
for key in ['길이', '높이', '두께']
    helper.regplot(train_df[key], 
                   train_df['무게(관측치)'],
                   train_df['무게(예측치)']
                   '훈련데이터', 
                   test_df[key], 
                   test_df['무게(관측치)'],
                   test_df['무게(예측치)'],
                   '검증데이터',
                   figsize=(15,5))

SyntaxError: invalid syntax (19865544.py, line 1)